# Installation and Model Loading

In [ ]:
%pip install unsloth
%pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install transformers datasets trl

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-jnrz29bi
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-jnrz29bi
  Resolved https://github.com/unslothai/unsloth.git to commit f48cb4133980650f32627c58666ef44d0f408b36
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.4-py3-none-any.whl size=190410 sha256=faf37670409f5c00d533b903107dad7fa65b6dd3333ad99e4081eded0ce36beb
  Stored in directory: /tmp/pip-ephem-wheel-cache-xq1y0_kz/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.3.4
    Uninstalling unsloth-2025.3.4:
      Successfully uninstalled unsloth-2025.3.4


In [2]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None      # Automatically choose optimal data type (e.g. float16/bfloat16)
load_in_4bit = True  # Enable 4-bit quantization for memory efficiency

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
    # token= HF_TOKEN
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.4: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config = None
)


Unsloth 2025.3.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [45]:
import json

ds_prompt = """
A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively,
i.e., <think> reasoning process here </think> <answer> answer here </answer>.

User: Analyze the sentiment and aspects in the following review text and output the labels in a json containing "aspect", "opinion", "polarity":
Review: {review}

Assistant:
<think>
Step 1: Parse the input text and extract the sentiment elements.
Step 2: Identify aspects, opinions, polarity, and categories based on the text.
Step 3: Structure the output as a JSON object with "labels" keys, matching the ABSA dataset format exactly.
</think>
<answer>
{answer}
</answer>

"""
EOS_TOKEN = tokenizer.eos_token
def format_absa_example(example, tokenizer):
    # Extract the review text.
    review = example["text"]
    labels = example["aspects"]
    # # Process each label.
    # # Replace "NULL" aspects with "general" (or another default if desired).
    # formatted_labels = []
    # for label in example["labels"]:
    #     aspect = label["aspect"] if label["aspect"] != "NULL" else "general"
    #     opinion = label["opinion"]
    #     polarity = label["polarity"]
    #     category = label["category"]
    #     formatted_labels.append(f"{aspect}: {opinion} ({polarity}, {category})")

    # # Join all formatted labels with a separator.
    # expected_output = "; ".join(formatted_labels)

    # Construct the prompt.

    prompt = ds_prompt.format(review = review, answer = f"'aspects': {labels['term']} 'sentiment': {labels['polarity']}")

    # Combine prompt and expected output, appending the end-of-sequence token.
    # full_text = prompt + " " + expected_output + tokenizer.eos_token
    full_text = prompt + EOS_TOKEN
    return {"text": full_text}


In [46]:
example1 = {'id': '2005',
 'text': 'it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous.',
 'aspects': {'term': ['quality', 'GUI', 'applications', 'use'],
  'polarity': ['positive', 'positive', 'positive', 'positive'],
  'from': [14, 36, 118, 143],
  'to': [21, 39, 130, 146]}}

In [47]:
print(format_absa_example(example1, tokenizer))

{'text': '\nA conversation between User and Assistant. The user asks a question, and the Assistant solves it.\nThe assistant first thinks about the reasoning process in the mind and then provides the user with the answer.\nThe reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively,\ni.e., <think> reasoning process here </think> <answer> answer here </answer>.\n\nUser: Analyze the sentiment and aspects in the following review text and output the labels in a json containing "aspect", "opinion", "polarity":\nReview: it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous.\n\nAssistant:\n<think>\nStep 1: Parse the input text and extract the sentiment elements.\nStep 2: Identify aspects, opinions, polarity, and categories based on the text.\nStep 3: Structure the output as a JSON object with "labels" keys, matching the AB

In [33]:
from datasets import load_dataset


# ds_train = load_dataset("json", data_files="laptop_quad_train.tsv.jsonl")
# ds_test = load_dataset("json", data_files="laptop_quad_test.tsv.jsonl")
ds_train = load_dataset("jakartaresearch/semeval-absa", "laptop", split = "train")
# Apply formatting (using batched map if desired)
# ds = ds.map(lambda examples: format_absa_example(examples, tokenizer), batched=False)
ds_train[3]

{'id': '2005',
 'text': 'it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous.',
 'aspects': {'term': ['quality', 'GUI', 'applications', 'use'],
  'polarity': ['positive', 'positive', 'positive', 'positive'],
  'from': [14, 36, 118, 143],
  'to': [21, 39, 130, 146]}}

In [48]:
ds_train = ds_train.map(lambda examples: format_absa_example(examples, tokenizer), batched=False)

Map:   0%|          | 0/3048 [00:00<?, ? examples/s]

In [51]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=200,  # Increase steps as needed for your dataset size
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs_absa",
    report_to="none"  # or "wandb" if you use Weights & Biases
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds_train,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # adjust as needed
    args=training_args,
)


Tokenizing to ["text"] (num_proc=2):   0%|          | 0/3048 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,048 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/5,383,329,280 (0.75% trained)


Step,Training Loss
10,0.638600
20,0.304100


In [29]:
 # For inference, set the model to inference mode
FastLanguageModel.for_inference(model)

# Example input – you can re-use the prompt format
sample_question = " Logic board utterly fried, cried, and laid down and died."
inputs = tokenizer([sample_question], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=200,
    use_cache=True,
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


 Logic board utterly fried, cried, and laid down and died. The user wants to understand the sentiment and aspects in the following review text and then output the labels and features in this order:
:
1. labels
2. features
3. output
based on:
1. In the 'Labels' field, list the labels you would extract from the text in this order:
   A. LAPTOP  
   B. DISPLAY  
   C. KEYBOARD  
   D. MOUSE  
   E. MOTHERBOARD  
   F. CPU  
   G. FANS&COOLING  
   H. PORTS  
   I. MEMORY  
   J. POWER_SUPPLY  
   K. OPTICAL_DRIVES  
   L. BATTERY  
   M. GRAPHICS  
   N. HARD_DISK  
   O. MULTIMEDIA_DEVICES  
   P. HARDWARE  
   Q. SOFTWARE  
   R. OS  
   S. WARRANTY  
   T. SHIPPING  
   U. SUPPORT  
   V. COMPANY  
   W. COMPANY  
  


In [28]:
ds_train['validation'][501]

{'id': '26:4',
 'text': '\nA conversation between User and Assistant. The user asks a question, and the Assistant solves it.\nThe assistant first thinks about the reasoning process in the mind and then provides the user with the answer.\nThe reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively,\ni.e., <think> reasoning process here </think> <answer> answer here </answer>.\n\nUser: Analyze the sentiment and aspects in the following review text and output the labels in this format:\n"labels": [\n  ``json\n    "aspect": <string extracted from the review representing the aspect>,\n    "opinion": <string extracted from the review representing the opinion>,\n    "polarity": <"positive" | "negative" | "neutral">,\n    "category": "<ENTITY>#<ATTRIBUTE>"\n  ``json,\n  ...\n]\nIn the output, the \'category\' field should be constructed by combining a valid ENTITY and a valid ATTRBIUTE using \'#\' as the delimiter. \nThe valid entity labels a

In [ ]:
model.save_pretrained("DeepSeek-R1-ABSA")
tokenizer.save_pretrained("DeepSeek-R1-ABSA")